<a href="https://colab.research.google.com/github/cedjustin/AI/blob/master/sprint13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# import dependencies
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
tf = tf.compat.v1
tf.disable_eager_execution()

[Problem 1] Looking back on the scratch
* had an epoch loop
* had to initialize weight and bias
* using mini batches
* have to calculate forward propagation and backward propagation
* using activation functions





[Problem 2] Consider the correspondence between scratch and TensorFlow

Using the numpy scratch vs tensorflow, tensorflow reduces the time of implementation and it gives more complexity to the things that you are implementing and which also results in giving you more control on what you are doing

【Problem 3】Create a model of Iris using all three object variables

In [6]:
class GetMiniBatch:
    """
    Iterator to get a mini-batch
    Parameters
    ----------
    X : The following forms of ndarray, shape (n_samples, n_features)
      Training data
    y : The following form of ndarray, shape (n_samples, 1)
      Correct answer value
    batch_size : int
      Batch size
    seed : int
      NumPy random number seed
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

In [7]:
def estimation_calc(X, Y, X_train, y_train, X_val, y_val, X_test, y_test, num_epochs, get_mini_batch_train):
  def example_net(x):
    """
    Simple 3-layer neural network
    """
    tf.random.set_random_seed(0)
    # Declaration of weight and bias
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.add and + are equivalent
    return layer_output
  #Read network structure                              
  logits = example_net(X)
  # Objective function
  loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
  # Optimization method
  optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
  train_op = optimizer.minimize(loss_op)
  # results
  pred = tf.nn.softmax(logits)
  # Estimated result
  correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(Y,1))
  #Indicator value calculation
  accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
  #Initialization of variable
  init = tf.global_variables_initializer()

  #Run calculation graph
  with tf.Session() as sess:
      sess.run(init)
      for epoch in range(num_epochs):
          #Loop for each epoch
          total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
          total_loss = 0
          total_acc = 0
          for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
              # Loop for each mini-batch
              sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
              loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
              total_loss += loss
          total_loss /= n_samples
          val_loss, acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
          # print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))
      test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
      print("test_acc : {:.3f}".format(test_acc))

Calculation using 3 species

In [8]:
#Load dataset
df = pd.read_csv("Iris.csv")
#Condition extraction from data frame
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
# NumPy 配列に変換
X = np.array(X)
y = np.array(y)
# Convert label to number
y[y == "Iris-setosa"] = 0
y[y == "Iris-versicolor"] = 1
y[y == "Iris-virginica"] = 2
y = y.astype(np.int64)[:, np.newaxis]

#Split into train and test
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train2, X_val2, y_train2, y_val2 = train_test_split(X_train2, y_train2, test_size=0.2, random_state=0)

# Hyperparameter settings
learning_rate = 0.001
batch_size = 10
num_epochs = 100
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train2.shape[1]
n_samples = X_train2.shape[0]
n_classes = 1
#Determine the shape of the argument to be passed to the calculation graph
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# train mini batch iterator
get_mini_batch_train2 = GetMiniBatch(X_train2, y_train2, batch_size=batch_size)
estimation_calc(X, Y, X_train2, y_train2, X_val2, y_val2, X_test2, y_test2, num_epochs, get_mini_batch_train2)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

test_acc : 1.000


Calculation using house prices

In [9]:
house_df = pd.read_csv("house_prices.csv")
house_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [10]:
h_X = house_df[['GrLivArea','YearBuilt']].to_numpy()
h_y = house_df['SalePrice'].to_numpy()
h_y = h_y[:,None]

In [11]:
h_X.shape, h_y.shape

((1460, 2), (1460, 1))

In [12]:
h_X_train, h_X_test, h_y_train, h_y_test = train_test_split(h_X, h_y, train_size = 0.2, random_state=0)
h_X_train, h_X_val, h_y_train, h_y_val = train_test_split(h_X_train, h_y_train, test_size=0.2, random_state=0)

In [64]:
# Hyperparameter settings
learning_rate = 0.001
batch_size = 10
num_epochs = 100
n_hidden1 = 50
n_hidden2 = 100
n_input = h_X_train.shape[1]
n_samples = h_X_train.shape[0]
n_classes = 1
#Determine the shape of the argument to be passed to the calculation graph
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# train mini batch iterator
h_get_mini_batch_train = GetMiniBatch(h_X_train, h_y_train, batch_size=batch_size)

In [72]:
def example_net(x):
  """
  Simple 3-layer neural network
  """
  tf.random.set_random_seed(0)
  # Declaration of weight and bias
  weights = {
      'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
      'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
      'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
  }
  biases = {
      'b1': tf.Variable(tf.random_normal([n_hidden1])),
      'b2': tf.Variable(tf.random_normal([n_hidden2])),
      'b3': tf.Variable(tf.random_normal([n_classes]))
  }
  layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
  layer_1 = tf.nn.relu(layer_1)
  layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
  layer_2 = tf.nn.relu(layer_2)
  layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.add and + are equivalent
  return layer_output
#Read network structure                              
logits = example_net(X)
# Objective function
loss_op =  tf.losses.mean_squared_error(labels=Y, predictions=logits)
# Optimization method
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# Estimated result
pred = logits
#Initialization of variable
init = tf.global_variables_initializer()

#Run calculation graph
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        #Loop for each epoch
        total_batch = np.ceil(h_X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = []
        for i, (mini_batch_x, mini_batch_y) in enumerate(h_get_mini_batch_train):
            # Loop for each mini-batch
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss = sess.run([loss_op], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss.append(loss)
        val_loss = sess.run([loss_op], feed_dict={X: h_X_val, Y: h_y_val})
    print("test_mse : {:.3f}".format(np.sum(total_loss)))

test_mse : 67193475072.000


[Problem 5] Creating a MNIST model

In [15]:
from keras.datasets import mnist
from sklearn.preprocessing import OneHotEncoder

In [54]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train.shape

(60000, 28, 28)

In [55]:
# flattening the subsets
X_train = X_train.reshape(-1,784)
X_test = X_test.reshape(-1,784)

In [56]:
# pre processing
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

In [57]:
# splitting our subsets into train and validation subsets
m_X_train, m_X_test, m_y_train, m_y_test = train_test_split(X_train, y_train, test_size=0.2, random_state= 0)

In [58]:
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(m_y_train[:, np.newaxis])
y_test_one_hot = enc.transform(m_y_test[:, np.newaxis])

In [61]:
# Hyperparameter settings
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = m_X_train.shape[1]
n_samples = m_X_train.shape[0]
n_classes = 10
#Determine the shape of the argument to be passed to the calculation graph
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# train mini batch iterator
m_get_mini_batch_train = GetMiniBatch(m_X_train, y_train_one_hot, batch_size=batch_size)

In [62]:
def example_net(x):
  """
  Simple 3-layer neural network
  """
  tf.random.set_random_seed(0)
  # Declaration of weight and bias
  weights = {
      'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
      'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
      'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
  }
  biases = {
      'b1': tf.Variable(tf.random_normal([n_hidden1])),
      'b2': tf.Variable(tf.random_normal([n_hidden2])),
      'b3': tf.Variable(tf.random_normal([n_classes]))
  }
  layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
  layer_1 = tf.nn.tanh(layer_1)
  layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
  layer_2 = tf.nn.tanh(layer_2)
  layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.add and + are equivalent
  return layer_output
#Read network structure                              
logits = example_net(X)
# Objective function
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
# Optimization method
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# Estimated result
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y,1))
#Indicator value calculation
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
#Initialization of variable
init = tf.global_variables_initializer()

#Run calculation graph
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        #Loop for each epoch
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(m_get_mini_batch_train):
            # Loop for each mini-batch
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        print("Epoch {}, loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, acc))
    test_acc = sess.run(accuracy, feed_dict={X: m_X_test, Y: y_test_one_hot})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 0.0810, acc : 0.900
Epoch 1, loss : 0.0404, acc : 1.000
Epoch 2, loss : 0.0355, acc : 1.000
Epoch 3, loss : 0.0357, acc : 0.900
Epoch 4, loss : 0.0325, acc : 0.800
Epoch 5, loss : 0.0334, acc : 1.000
Epoch 6, loss : 0.0309, acc : 0.900
Epoch 7, loss : 0.0296, acc : 1.000
Epoch 8, loss : 0.0291, acc : 0.900
Epoch 9, loss : 0.0289, acc : 0.700
test_acc : 0.894
